In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
%run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [3]:
# %run ../../../prepare_data.py
X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_syllable_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
save_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', X_train, y_train, other_features = X_other_features_train)
save_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', X_test, y_test, other_features = X_other_features_test)
save_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', other_features=True)

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
CREATING SYLLABLE DICTIONARY...
CREATION SUCCESSFUL!
GENERATING X AND y...
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
GENERATION SUCCESSFUL!


In [77]:
%run ../../../prepare_data.py
gen = generate_fake_epoch_syllables(X_train[:20], X_other_features_train[:20], y_train[:20], 16, 5168)

In [85]:
# prints test word from generator
# print(X_train[0])
# print(syllable_dictionary[test[0][0][0][0].tolist().index(1)])
print(np.array(test[0][0]).shape)
for el in test[0][0][0]:
    print(el)
    print(syllable_dictionary[el.tolist().index(1)])

(4, 11, 5168)
[0 0 0 ..., 0 0 0]
bolj
[0 0 0 ..., 0 0 0]
ši
[0 0 0 ..., 0 0 0]
mi
[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]

[1 0 0 ..., 0 0 0]



In [84]:
# print(X_train[0])
test = next(gen)
# eye = np.eye(5168)[X_train[0]]
# print(X_train[0].tolist())
# np.eye(5168)[[2, 1]]



In [123]:
np.eye(10)[[3,7,2,5]]
print(eye[[2, 3]].shape)

(2, 5168)


In [11]:
X_validate.shape
# len(syllable_dictionary)

(54222, 11)

In [46]:
print(len(syllable_dictionary))
shuffle_vector_location = '../../internal_representations/inputs/shuffle_vector'
X_train, X_other_features_train, y_train = generate_syllable_X_and_y(syllable_dictionary, max_word, max_num_vowels, train_content, vowels, accetuated_vowels, feature_dictionary, shuffle_vector_location + '_train.h5')


5168
CREATING OTHER FEATURES...
OTHER FEATURES CREATED!
SHUFFELING INPUTS...
(430151, 11)
(430151,)
INPUTS SHUFFELED!


In [5]:
print (X_train.shape)
print (X_test.shape)
print (X_validate.shape)
# X_other_features = create_X_features(content)
# print (X_other_features[178200])
decode_position = 38
print (decode_input(X_validate[decode_position], dictionary))
# print (X_other_features_train[0])
print (decode_X_features(feature_dictionary, [X_other_features_train[decode_position]]))
# print (len(X_other_features_train[0]))

(430151, 23, 30)
(52058, 23, 30)
(54222, 23, 30)


NameError: name 'dictionary' is not defined

In [9]:
num_examples = len(X_train) # training set size
# nn_input_dim = max_word * len(dictionary) # input layer dimensionality
# nn_output_dim = max_num_vowels * max_num_vowels # output layer dimensionality
nn_output_dim = 11
nn_hdim = 516
batch_size = 16
actual_epoch = 20
num_fake_epoch = 20

# Gradient descent parameters (I picked these by hand)
# epsilon = 1 # learning rate for gradient descent
# reg_lambda = 1 # regularization strength

In [11]:
conv_input_shape=(11, 5168)
othr_input = (140, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
x_conv = Conv1D(22, (2), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(22, (2), padding='same', activation='relu')(conv_input)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)

In [12]:
model = Model(inputs=[conv_input, othr_input], outputs=x)

# epochs = 5
# lrate = 0.1
# decay = lrate/epochs
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Compile model
# keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [93]:
# history = model.fit_generator(generate_fake_epoch_syllables(X_test[:1000], X_other_features_test[:1000], y_test[:1000], batch_size), X_test[:1000].shape[0]/(batch_size * num_fake_epoch), epochs=actual_epoch*num_fake_epoch, validation_data=generate_fake_epoch_syllables(X_test[:1000], X_other_features_test[:1000], y_test[:1000], batch_size), validation_steps=X_test[:1000].shape[0]/(batch_size * num_fake_epoch))

In [13]:
# history = model.fit_generator(generate_fake_epoch(X_train, X_other_features_train, y_train, batch_size), X_train.shape[0]/(batch_size * num_fake_epoch), epochs=actual_epoch*num_fake_epoch, validation_data=([X_test, X_other_features_test], y_test))
history = model.fit_generator(generate_fake_epoch_syllables(X_train, X_other_features_train, y_train, batch_size), X_train.shape[0]/(batch_size * num_fake_epoch), epochs=actual_epoch*num_fake_epoch, validation_data=generate_fake_epoch_syllables(X_test, X_other_features_test, y_test, batch_size), validation_steps=X_test.shape[0]/(batch_size * num_fake_epoch))


# model.fit([X_train, X_other_features_train], y_train, validation_data=([X_validate, X_other_features_validate], y_validate), epochs=1, batch_size=16)

Epoch 1/400
1345/1344 [==============================] - 111s - loss: 0.2137 - actual_accuracy: 0.2185 - val_loss: 0.1504 - val_actual_accuracy: 0.4659
Epoch 2/400
1345/1344 [==============================] - 111s - loss: 0.1296 - actual_accuracy: 0.5341 - val_loss: 0.1084 - val_actual_accuracy: 0.6798
Epoch 3/400
1345/1344 [==============================] - 125s - loss: 0.1025 - actual_accuracy: 0.6819 - val_loss: 0.0947 - val_actual_accuracy: 0.7362
Epoch 4/400
1345/1344 [==============================] - 136s - loss: 0.0882 - actual_accuracy: 0.7435 - val_loss: 0.0897 - val_actual_accuracy: 0.7615
Epoch 5/400
1345/1344 [==============================] - 130s - loss: 0.0786 - actual_accuracy: 0.7798 - val_loss: 0.0883 - val_actual_accuracy: 0.7676
Epoch 6/400
1345/1344 [==============================] - 144s - loss: 0.0697 - actual_accuracy: 0.8061 - val_loss: 0.0821 - val_actual_accuracy: 0.7849
Epoch 7/400
1345/1344 [==============================] - 141s - loss: 0.0641 - actual_ac

1345/1344 [==============================] - 143s - loss: 0.0154 - actual_accuracy: 0.9633 - val_loss: 0.1348 - val_actual_accuracy: 0.8378
Epoch 54/400
1345/1344 [==============================] - 144s - loss: 0.0164 - actual_accuracy: 0.9612 - val_loss: 0.1256 - val_actual_accuracy: 0.8439
Epoch 55/400
1345/1344 [==============================] - 146s - loss: 0.0168 - actual_accuracy: 0.9618 - val_loss: 0.1161 - val_actual_accuracy: 0.8551
Epoch 56/400
1345/1344 [==============================] - 144s - loss: 0.0162 - actual_accuracy: 0.9616 - val_loss: 0.1558 - val_actual_accuracy: 0.8416
Epoch 57/400
1345/1344 [==============================] - 142s - loss: 0.0161 - actual_accuracy: 0.9625 - val_loss: 0.1309 - val_actual_accuracy: 0.8463
Epoch 58/400
1345/1344 [==============================] - 144s - loss: 0.0161 - actual_accuracy: 0.9628 - val_loss: 0.1362 - val_actual_accuracy: 0.8344
Epoch 59/400
1345/1344 [==============================] - 141s - loss: 0.0162 - actual_accuracy

1345/1344 [==============================] - 145s - loss: 0.0104 - actual_accuracy: 0.9776 - val_loss: 0.1430 - val_actual_accuracy: 0.8528
Epoch 107/400
1345/1344 [==============================] - 149s - loss: 0.0098 - actual_accuracy: 0.9779 - val_loss: 0.1644 - val_actual_accuracy: 0.8436
Epoch 108/400
1345/1344 [==============================] - 147s - loss: 0.0100 - actual_accuracy: 0.9772 - val_loss: 0.1502 - val_actual_accuracy: 0.8474
Epoch 109/400
1345/1344 [==============================] - 145s - loss: 0.0095 - actual_accuracy: 0.9788 - val_loss: 0.1543 - val_actual_accuracy: 0.8493TA: 3s - l
Epoch 110/400
1345/1344 [==============================] - 150s - loss: 0.0107 - actual_accuracy: 0.9748 - val_loss: 0.1465 - val_actual_accuracy: 0.8512
Epoch 111/400
1345/1344 [==============================] - 152s - loss: 0.0098 - actual_accuracy: 0.9766 - val_loss: 0.1597 - val_actual_accuracy: 0.8528
Epoch 112/400
1345/1344 [==============================] - 141s - loss: 0.0095 -

1345/1344 [==============================] - 160s - loss: 0.0079 - actual_accuracy: 0.9828 - val_loss: 0.1667 - val_actual_accuracy: 0.8520
Epoch 160/400
1345/1344 [==============================] - 150s - loss: 0.0077 - actual_accuracy: 0.9831 - val_loss: 0.1868 - val_actual_accuracy: 0.8432
Epoch 161/400
1345/1344 [==============================] - 146s - loss: 0.0078 - actual_accuracy: 0.9828 - val_loss: 0.1706 - val_actual_accuracy: 0.8386
Epoch 162/400
1345/1344 [==============================] - 151s - loss: 0.0078 - actual_accuracy: 0.9828 - val_loss: 0.1901 - val_actual_accuracy: 0.8447
Epoch 163/400
1345/1344 [==============================] - 148s - loss: 0.0071 - actual_accuracy: 0.9837 - val_loss: 0.1894 - val_actual_accuracy: 0.8355
Epoch 164/400
1345/1344 [==============================] - 154s - loss: 0.0073 - actual_accuracy: 0.9836 - val_loss: 0.1782 - val_actual_accuracy: 0.8577
Epoch 165/400
1345/1344 [==============================] - 154s - loss: 0.0071 - actual_ac

1345/1344 [==============================] - 167s - loss: 0.0063 - actual_accuracy: 0.9861 - val_loss: 0.2013 - val_actual_accuracy: 0.8447
Epoch 213/400
1345/1344 [==============================] - 170s - loss: 0.0056 - actual_accuracy: 0.9881 - val_loss: 0.2015 - val_actual_accuracy: 0.8470
Epoch 214/400
1345/1344 [==============================] - 170s - loss: 0.0062 - actual_accuracy: 0.9866 - val_loss: 0.2123 - val_actual_accuracy: 0.8367
Epoch 215/400
1345/1344 [==============================] - 168s - loss: 0.0063 - actual_accuracy: 0.9856 - val_loss: 0.1813 - val_actual_accuracy: 0.8520
Epoch 216/400
1345/1344 [==============================] - 173s - loss: 0.0060 - actual_accuracy: 0.9864 - val_loss: 0.1902 - val_actual_accuracy: 0.8520
Epoch 217/400
1345/1344 [==============================] - 170s - loss: 0.0058 - actual_accuracy: 0.9873 - val_loss: 0.1817 - val_actual_accuracy: 0.8459
Epoch 218/400
1345/1344 [==============================] - 161s - loss: 0.0058 - actual_ac

1345/1344 [==============================] - 172s - loss: 0.0050 - actual_accuracy: 0.9887 - val_loss: 0.2021 - val_actual_accuracy: 0.8459
Epoch 266/400
1345/1344 [==============================] - 173s - loss: 0.0048 - actual_accuracy: 0.9903 - val_loss: 0.2139 - val_actual_accuracy: 0.8439
Epoch 267/400
1345/1344 [==============================] - 176s - loss: 0.0053 - actual_accuracy: 0.9879 - val_loss: 0.1784 - val_actual_accuracy: 0.8673
Epoch 268/400
1345/1344 [==============================] - 173s - loss: 0.0046 - actual_accuracy: 0.9903 - val_loss: 0.1895 - val_actual_accuracy: 0.8551
Epoch 269/400
1345/1344 [==============================] - 173s - loss: 0.0050 - actual_accuracy: 0.9893 - val_loss: 0.2127 - val_actual_accuracy: 0.8497
Epoch 270/400
1345/1344 [==============================] - 172s - loss: 0.0053 - actual_accuracy: 0.9880 - val_loss: 0.1972 - val_actual_accuracy: 0.8535
Epoch 271/400
1345/1344 [==============================] - 170s - loss: 0.0051 - actual_ac

1345/1344 [==============================] - 213s - loss: 0.0044 - actual_accuracy: 0.9910 - val_loss: 0.2065 - val_actual_accuracy: 0.8547
Epoch 319/400
1345/1344 [==============================] - 210s - loss: 0.0048 - actual_accuracy: 0.9893 - val_loss: 0.2231 - val_actual_accuracy: 0.8470
Epoch 320/400
1345/1344 [==============================] - 281s - loss: 0.0051 - actual_accuracy: 0.9885 - val_loss: 0.2090 - val_actual_accuracy: 0.8551
Epoch 321/400
1345/1344 [==============================] - 184s - loss: 0.0046 - actual_accuracy: 0.9886 - val_loss: 0.2119 - val_actual_accuracy: 0.8535
Epoch 322/400
1345/1344 [==============================] - 195s - loss: 0.0046 - actual_accuracy: 0.9893 - val_loss: 0.2403 - val_actual_accuracy: 0.8394
Epoch 323/400
1345/1344 [==============================] - 216s - loss: 0.0045 - actual_accuracy: 0.9903 - val_loss: 0.2013 - val_actual_accuracy: 0.8554
Epoch 324/400
1345/1344 [==============================] - 221s - loss: 0.0043 - actual_ac

1345/1344 [==============================] - 165s - loss: 0.0043 - actual_accuracy: 0.9911 - val_loss: 0.2193 - val_actual_accuracy: 0.8570
Epoch 372/400
1345/1344 [==============================] - 168s - loss: 0.0045 - actual_accuracy: 0.9905 - val_loss: 0.2042 - val_actual_accuracy: 0.8627
Epoch 373/400
1345/1344 [==============================] - 167s - loss: 0.0035 - actual_accuracy: 0.9919 - val_loss: 0.2110 - val_actual_accuracy: 0.8512
Epoch 374/400
1345/1344 [==============================] - 163s - loss: 0.0041 - actual_accuracy: 0.9909 - val_loss: 0.2205 - val_actual_accuracy: 0.8459
Epoch 375/400
1345/1344 [==============================] - 169s - loss: 0.0042 - actual_accuracy: 0.9907 - val_loss: 0.2059 - val_actual_accuracy: 0.8597
Epoch 376/400
1345/1344 [==============================] - 169s - loss: 0.0043 - actual_accuracy: 0.9904 - val_loss: 0.1887 - val_actual_accuracy: 0.8608
Epoch 377/400
1345/1344 [==============================] - 164s - loss: 0.0035 - actual_ac

In [8]:
name = '20_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [46]:
dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
train_content, test_content, validate_content = split_content(content, 0.2, '../../internal_representations/inputs/content_shuffle_vector.h5')
feature_dictionary = create_feature_dictionary()
syllable_dictionary = create_syllables_dictionary(content, vowels)
def generate_input_from_word(word, max_word, dictionary):
    x = np.zeros((max_word, len(dictionary)))
    j = 0
    for c in list(word):
        index = 0
        for d in dictionary:
            if c == d:
                x[j, index] = 1
                break
            index += 1
        j += 1
    return x

# model = load_model()
# prediction = predict(model, generate_input_from_word('hidrija'))
# print decode_position(prediction[0])

READING CONTENT...
CONTENT READ SUCCESSFULY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!


In [5]:
# create_syllables(content[375488][0], vowels)
create_syllables('izzvati', vowels)

['iz', 'zva', 'ti']

In [45]:

syllables_dictionary = create_syllables_dictionary(content, vowels)

NameError: name 'content' is not defined

In [9]:
syllables_dictionary = sorted(syllables_dictionary)
syllables_dictionary

['',
 '-o',
 'a',
 'ab',
 'ah',
 'akt',
 'al',
 'alf',
 'alg',
 'alt',
 'am',
 'an',
 'ant',
 'ar',
 'arh',
 'as',
 'at',
 'av',
 'ba',
 'bab',
 'bah',
 'baj',
 'bajt',
 'bak',
 'bal',
 'bam',
 'ban',
 'banj',
 'bank',
 'bar',
 'bark',
 'barv',
 'bas',
 'bast',
 'bat',
 'bav',
 'baz',
 'bač',
 'baš',
 'baž',
 'bde',
 'bdel',
 'bdet',
 'bdeč',
 'bdi',
 'bdim',
 'bdiš',
 'be',
 'bec',
 'bed',
 'beg',
 'beh',
 'bej',
 'bek',
 'bel',
 'belc',
 'belj',
 'belk',
 'bem',
 'ben',
 'benj',
 'benk',
 'bent',
 'ber',
 'bes',
 'bet',
 'bev',
 'bez',
 'beč',
 'beš',
 'bež',
 'bi',
 'bic',
 'bih',
 'bij',
 'bik',
 'bil',
 'bilj',
 'bilk',
 'bim',
 'bin',
 'bir',
 'bisk',
 'bit',
 'bitk',
 'biv',
 'bič',
 'biš',
 'bišč',
 'bja',
 'bjav',
 'bje',
 'bjej',
 'bjekt',
 'bjel',
 'bjem',
 'bjest',
 'bjet',
 'bješ',
 'bji',
 'bjih',
 'bjim',
 'bjo',
 'bju',
 'bla',
 'blag',
 'blah',
 'blak',
 'blam',
 'blan',
 'blast',
 'blat',
 'ble',
 'bled',
 'blej',
 'blek',
 'blem',
 'blesk',
 'blest',
 'blet',
 'bleč'

In [57]:
%run ../../../prepare_data.py
# generate_X_and_y(dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels, feature_dictionary)